# Multi-Agent Service Matcher

## Overview

This notebook demonstrates a **simple multi-agent system** that matches a user's free-text request to the best service from a small list of offerings. The system uses **three core agentic concepts**:

1. **Multi-Agent Pipeline** - A sequential chain of three specialized LLM-powered agents
2. **Custom Tool** - A function that provides data to agents
3. **In-Memory Session State** - A mechanism for agents to share context and pass information forward

---

## What You'll Learn

- ✅ How to design a **sequential multi-agent pipeline** where each agent has a specific role
- ✅ How to implement **custom tools** that agents can use to access data
- ✅ How to manage **session state** so agents can communicate and build on each other's work
- ✅ How to use **natural language reasoning** for matching instead of keywords or embeddings
- ✅ How to structure prompts for different agent roles

---

## System Architecture

```
User Request
     |
     v
┌─────────────────────────┐
│  AGENT 1: INTERPRETER   │  Role: Clarify and normalize the user's request
│  Input: Raw user text   │
│  Output: Clear summary  │
└──────────┬──────────────┘
           │ (writes to session)
           v
┌─────────────────────────┐
│  AGENT 2: MATCHER       │  Role: Find the best matching service
│  Input: Summary + Tool  │  Tool: get_services() → returns service list
│  Output: Best match     │
└──────────┬──────────────┘
           │ (writes to session)
           v
┌─────────────────────────┐
│  AGENT 3: POLISHER      │  Role: Format user-friendly final answer
│  Input: Matched service │
│  Output: Final response │
└──────────┬──────────────┘
           │
           v
    Final Answer
```

---

## Why This Architecture?

**Separation of Concerns**: Each agent has ONE job, making the system:
- Easier to debug (you can inspect each step)
- More maintainable (you can improve one agent without touching others)
- More testable (you can test each agent independently)

**Sequential Pipeline**: Information flows in one direction, making it:
- Simple to understand and reason about
- Predictable in execution
- Easy to extend (just add more agents to the chain)

---

## 1. Setup and Installation

We'll use **Google Vertex AI** for our LLM calls. Vertex AI is Google Cloud's enterprise AI platform that provides access to Gemini models through a managed service.

### Why Vertex AI?
- Enterprise-grade security and reliability
- Integrated with Google Cloud Project
- No separate API key needed (uses your Google Cloud credentials)
- Better for production use cases

In [1]:
# Dependencies are managed via requirements.txt
# Install them once in your terminal before running this notebook:
#   pip install -r requirements.txt
#
# If running in Google Colab, uncomment the line below:
# !pip install -q google-cloud-aiplatform python-dotenv

In [2]:
import json
import os
from typing import Dict, List, Any
from dotenv import load_dotenv
from google import genai

# Load environment variables from .env file
load_dotenv()

# Get configuration from environment variables
PROJECT_ID = os.getenv("GOOGLE_CLOUD_PROJECT")
LOCATION = os.getenv("GOOGLE_CLOUD_LOCATION", "us-central1")

print("="*60)
print("🔧 Google GenAI Configuration")
print("="*60)
print(f"Project ID: {PROJECT_ID}")
print(f"Location: {LOCATION}")
print()

try:
    # Initialize Google GenAI client with Vertex AI
    client = genai.Client(vertexai=True, project=PROJECT_ID, location=LOCATION)
    
    # Test with a simple query to verify it works
    print("Testing connection...")
    test_response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents="Say 'Ready!' if you can read this"
    )
    print(f"✅ Google GenAI initialized successfully!")
    print(f"✅ Model response: {test_response.text.strip()}")
    print()
    print("Ready to use Gemini models via Vertex AI")
except Exception as e:
    print("❌ Failed to initialize Google GenAI")
    print(f"Error: {e}")
    print()
    print("Please ensure:")
    print("  1. You've run: gcloud auth application-default login")
    print("  2. Your project has Vertex AI API enabled")
    print("  3. You have the correct permissions")

🔧 Google GenAI Configuration
Project ID: d-ulti-ml-ds-dev-9561
Location: us-central1

Testing connection...
✅ Google GenAI initialized successfully!
✅ Model response: Ready!

Ready to use Gemini models via Vertex AI


---

## 2. Custom Tool: Service Database

### What is a Custom Tool?

In agentic systems, a **tool** is a function that an agent can call to:
- Access external data (databases, APIs, files)
- Perform calculations or transformations
- Execute actions (send emails, update records)

**Why use tools?**
- LLMs can't access real-time data on their own
- Tools extend what agents can do beyond text generation
- Tools provide structured, reliable data (unlike asking an LLM to "remember" information)

### Our Tool: `get_services()`

This simple tool returns a hardcoded list of services. In a real application, this might:
- Query a database
- Call an API
- Load from a file

For learning purposes, we keep it simple with in-memory data.

In [3]:
def get_services() -> List[Dict[str, Any]]:
    """
    Custom tool that returns the list of available services.
    
    This is a TOOL because:
    - It provides data that agents need but don't have
    - It's a reusable function that could be called by any agent
    - It separates data from logic (agents focus on reasoning, tools provide data)
    
    Returns:
        List of service dictionaries, each containing:
        - service_id: Unique identifier
        - service_title: Name of the service
        - service_description: What the service does
    """
    
    # Hardcoded service database
    # In a real system, this might come from:
    # - A SQL database: SELECT * FROM services
    # - A REST API: requests.get("https://api.example.com/services")
    # - A JSON file: json.load(open("services.json"))
    
    services = [
        {
            "service_id": 1,
            "service_title": "Python Debug Helper",
            "service_description": "I help fix Python bugs quickly using simple explanations."
        },
        {
            "service_id": 2,
            "service_title": "Tax Filing Advisor",
            "service_description": "I answer questions about government tax forms and common filing issues."
        },
        {
            "service_id": 3,
            "service_title": "Swimming Technique Review",
            "service_description": "I give fast feedback on stroke mechanics, breathing, and body position."
        },
        {
            "service_id": 4,
            "service_title": "SAT Math Tutor",
            "service_description": "I explain SAT math problems step-by-step and help improve accuracy."
        }
    ]
    
    return services


# Test the tool
print("🔧 Testing get_services() tool:\n")
services = get_services()
for service in services:
    print(f"  [{service['service_id']}] {service['service_title']}")
    print(f"      → {service['service_description']}")
    print()

print(f"✓ Tool returns {len(services)} services")

🔧 Testing get_services() tool:

  [1] Python Debug Helper
      → I help fix Python bugs quickly using simple explanations.

  [2] Tax Filing Advisor
      → I answer questions about government tax forms and common filing issues.

  [3] Swimming Technique Review
      → I give fast feedback on stroke mechanics, breathing, and body position.

  [4] SAT Math Tutor
      → I explain SAT math problems step-by-step and help improve accuracy.

✓ Tool returns 4 services


---

## 3. Session State Management

### What is Session State?

**Session state** is a shared memory space where agents store and retrieve information during a single execution run.

**Why do we need it?**
- Agents need to **communicate**: Agent 2 needs Agent 1's output
- Agents need **context**: Each agent builds on previous work
- We need **transparency**: We can inspect what each agent produced

### Our Implementation: Python Dictionary

We use a simple Python dictionary as our session state:

```python
session = {
    "user_request": "original request",           # Input from user
    "clarified_request": "...",                   # Agent 1 writes this
    "matched_service": {...},                      # Agent 2 writes this
    "final_response": "..."                       # Agent 3 writes this
}
```

**Key Principle**: Each agent:
1. Reads what it needs from the session
2. Does its work (usually an LLM call)
3. Writes its output back to the session
4. Returns the session for the next agent

This creates a "pipeline" where information flows forward.

In [4]:
def create_session(user_request: str) -> Dict[str, Any]:
    """
    Initialize a new session with the user's request.
    
    The session is a dictionary that will be passed through the pipeline.
    Each agent will read from it and write to it.
    
    Args:
        user_request: The raw text from the user
        
    Returns:
        A new session dictionary with the user_request as the starting point
    """
    return {
        "user_request": user_request,      # What the user originally asked for
        "clarified_request": None,          # Agent 1 will fill this
        "matched_service": None,            # Agent 2 will fill this
        "final_response": None              # Agent 3 will fill this
    }


def print_session_state(session: Dict[str, Any], title: str = "Session State"):
    """
    Helper function to inspect the current session state.
    Useful for debugging and learning how data flows through the pipeline.
    """
    print(f"\n{'='*60}")
    print(f"  {title}")
    print(f"{'='*60}")
    for key, value in session.items():
        print(f"\n{key}:")
        if isinstance(value, dict):
            print(json.dumps(value, indent=2))
        else:
            print(f"  {value}")
    print(f"\n{'='*60}\n")


# Test session creation
test_session = create_session("I need help with my Python code")
print_session_state(test_session, "Example: Fresh Session")


  Example: Fresh Session

user_request:
  I need help with my Python code

clarified_request:
  None

matched_service:
  None

final_response:
  None




---

## 4. Agent 1: The Interpreter

### Role
The Interpreter takes messy, informal user input and transforms it into a clear, normalized summary.

### Why is this important?
- Users write casually: "my kid needs help w/ sat math probs" 
- Later agents work better with clean input: "User needs SAT math tutoring"
- It removes noise and focuses on the core need

### How it works
1. Read `user_request` from session
2. Send it to the LLM with a specific prompt
3. Store the LLM's clarified version in `clarified_request`
4. Return the updated session

### Prompt Engineering Note
The prompt tells the LLM:
- Its role ("You are an interpreter")
- What to do ("rewrite into a clear summary")
- Constraints ("one or two sentences", "don't add information")

In [5]:
def agent_1_interpreter(session: Dict[str, Any]) -> Dict[str, Any]:
    """
    AGENT 1: INTERPRETER
    
    Clarifies and normalizes the user's raw request into a clean summary.
    
    Input (from session):
        - user_request: Raw text from the user
        
    Output (written to session):
        - clarified_request: A clear, normalized summary of what the user needs
        
    Returns:
        Updated session dictionary
    """
    
    print("\n🤖 AGENT 1: INTERPRETER is working...")
    
    # Step 1: Read input from session
    user_request = session["user_request"]
    print(f"   Reading user request: '{user_request}'")
    
    # Step 2: Prepare the prompt for the LLM
    # This is where we define the agent's "personality" and task
    prompt = f"""
You are an interpreter agent. Your job is to take a user's informal, possibly messy request and rewrite it into a clear, concise summary.

Rules:
1. Keep it to one or two sentences
2. Focus on the main need or problem
3. Remove filler words and casual language
4. Don't add information that wasn't in the original request
5. Don't try to solve the problem - just clarify what they're asking for

User request: "{user_request}"

Provide only the clarified summary, nothing else.
"""
    
    # Step 3: Call the LLM using Google GenAI
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=prompt
    )
    clarified_request = response.text.strip()
    
    print(f"   ✓ Clarified to: '{clarified_request}'")
    
    # Step 4: Write output to session
    session["clarified_request"] = clarified_request
    
    # Step 5: Return the updated session for the next agent
    return session


# Test Agent 1 in isolation
print("\n" + "="*60)
print("Testing Agent 1: Interpreter")
print("="*60)

test_session = create_session("my python loop keeps skipping stuff idk why")
test_session = agent_1_interpreter(test_session)

print("\nResult:")
print(f"  Original: {test_session['user_request']}")
print(f"  Clarified: {test_session['clarified_request']}")


Testing Agent 1: Interpreter

🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'my python loop keeps skipping stuff idk why'
   ✓ Clarified to: 'I need help understanding why my Python loop is skipping iterations.'

Result:
  Original: my python loop keeps skipping stuff idk why
  Clarified: I need help understanding why my Python loop is skipping iterations.


---

## 5. Agent 2: The Matcher

### Role
The Matcher finds the best service for the user's clarified request using natural language reasoning.

### Why is this the hardest agent?
- It must **use the tool** (call `get_services()`) to get data
- It must **reason** about which service best matches the request
- It must **explain** its choice
- It must return **structured data** (JSON) that the next agent can use

### How it works
1. Read `clarified_request` from session
2. Call the `get_services()` tool to get the list of services
3. Send both the request and service list to the LLM
4. Ask the LLM to pick the best match and explain why
5. Parse the LLM's response as JSON
6. Store the matched service in `matched_service`
7. Return the updated session

### Key Learning: Tool Integration
Notice how we:
1. Call the tool ourselves (not the LLM)
2. Include the tool's output in the prompt
3. Let the LLM reason over the data

This is **tool-augmented generation**: we give the LLM real data to work with.

In [6]:
def agent_2_matcher(session: Dict[str, Any]) -> Dict[str, Any]:
    """
    AGENT 2: MATCHER
    
    Finds the best matching service by:
    1. Calling the get_services() tool
    2. Using LLM reasoning to compare the user's need against each service
    3. Selecting the single best match
    4. Explaining why it's the best match
    
    Input (from session):
        - clarified_request: The normalized user request from Agent 1
        
    Output (written to session):
        - matched_service: Dictionary containing:
            - service_id, service_title, service_description
            - reason: Why this service was chosen
        
    Returns:
        Updated session dictionary
    """
    
    print("\n🤖 AGENT 2: MATCHER is working...")
    
    # Step 1: Read input from session
    clarified_request = session["clarified_request"]
    print(f"   Reading clarified request: '{clarified_request}'")
    
    # Step 2: Call the custom tool to get services
    # This is the key "tool use" concept - agents can call functions to get data
    print("   Calling get_services() tool...")
    services = get_services()
    print(f"   ✓ Retrieved {len(services)} services")
    
    # Step 3: Format services for the LLM
    # We convert the list of services into a readable format
    services_text = "\n".join([
        f"Service {s['service_id']}: {s['service_title']}\n  Description: {s['service_description']}"
        for s in services
    ])
    
    # Step 4: Prepare the prompt
    # Notice how we provide both the user's need AND the service list
    prompt = f"""
You are a matcher agent. Your job is to find the single best matching service for a user's request.

User's need: "{clarified_request}"

Available services:
{services_text}

Instructions:
1. Read the user's need carefully
2. Compare it against each service's title and description
3. Use reasoning to determine which service is the BEST match
4. Provide a short explanation of WHY this service matches best

Return your answer as JSON with this exact structure:
{{
  "service_id": <number>,
  "service_title": "<exact title>",
  "service_description": "<exact description>",
  "reason": "<your explanation of why this is the best match>"
}}

Return ONLY the JSON, no other text.
"""
    
    # Step 5: Call the LLM using Google GenAI
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=prompt
    )
    
    # Extract JSON from response (sometimes LLMs add markdown code blocks)
    response_text = response.text.strip()
    if response_text.startswith("```"):
        # Remove markdown code blocks if present
        response_text = response_text.split("```")[1]
        if response_text.startswith("json"):
            response_text = response_text[4:]
    matched_service = json.loads(response_text.strip())
    
    print(f"   ✓ Matched to: {matched_service['service_title']}")
    print(f"   Reason: {matched_service['reason']}")
    
    # Step 6: Write output to session
    session["matched_service"] = matched_service
    
    # Step 7: Return the updated session
    return session


# Test Agent 2 in isolation
print("\n" + "="*60)
print("Testing Agent 2: Matcher")
print("="*60)

test_session = create_session("I need help with Python loops")
test_session["clarified_request"] = "User needs help debugging Python for-loops"
test_session = agent_2_matcher(test_session)

print("\nResult:")
print(json.dumps(test_session["matched_service"], indent=2))


Testing Agent 2: Matcher

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'User needs help debugging Python for-loops'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Python Debug Helper
   Reason: The user needs help debugging Python code, specifically for-loops, and the Python Debug Helper service is designed to help fix Python bugs.

Result:
{
  "service_id": 1,
  "service_title": "Python Debug Helper",
  "service_description": "I help fix Python bugs quickly using simple explanations.",
  "reason": "The user needs help debugging Python code, specifically for-loops, and the Python Debug Helper service is designed to help fix Python bugs."
}


---

## 6. Agent 3: The Polisher

### Role
The Polisher takes the matched service and creates a polished, user-friendly final response.

### Why do we need this agent?
- Agent 2 returns structured data (JSON) - not user-friendly
- We want a natural, conversational final answer
- This separates "finding the match" from "presenting the match"

### How it works
1. Read `matched_service` from session
2. Send it to the LLM with formatting instructions
3. Store the formatted response in `final_response`
4. Return the updated session

### Design Principle: Single Responsibility
Notice how each agent does ONE thing:
- Agent 1: Clarify
- Agent 2: Match
- Agent 3: Format

We could combine these, but keeping them separate makes the system:
- Easier to modify (change formatting without touching matching logic)
- Easier to debug (see exactly where things go wrong)
- More reusable (swap out any agent without affecting others)

In [7]:
def agent_3_polisher(session: Dict[str, Any]) -> Dict[str, Any]:
    """
    AGENT 3: POLISHER
    
    Creates a user-friendly final response from the matched service.
    
    Input (from session):
        - matched_service: The service selected by Agent 2, with reason
        
    Output (written to session):
        - final_response: A polished, conversational answer for the user
        
    Returns:
        Updated session dictionary
    """
    
    print("\n🤖 AGENT 3: POLISHER is working...")
    
    # Step 1: Read input from session
    matched_service = session["matched_service"]
    print(f"   Reading matched service: {matched_service['service_title']}")
    
    # Step 2: Prepare the prompt
    # We give the LLM the matched service data and ask for clean formatting
    prompt = f"""
You are a polisher agent. Your job is to create a friendly, clear final response for the user.

You have matched the user to this service:
- Title: {matched_service['service_title']}
- Description: {matched_service['service_description']}
- Reason for match: {matched_service['reason']}

Create a response with this structure:

Best Match: [service title]
Description: [service description]
Why: [explain why this service matches the user's need]

Keep it concise and friendly. The "Why" should be conversational and clear.
"""
    
    # Step 3: Call the LLM using Google GenAI
    response = client.models.generate_content(
        model='gemini-2.0-flash-exp',
        contents=prompt
    )
    final_response = response.text.strip()
    
    print("   ✓ Final response created")
    
    # Step 4: Write output to session
    session["final_response"] = final_response
    
    # Step 5: Return the updated session
    return session


# Test Agent 3 in isolation
print("\n" + "="*60)
print("Testing Agent 3: Polisher")
print("="*60)

test_session = create_session("test")
test_session["matched_service"] = {
    "service_id": 4,
    "service_title": "SAT Math Tutor",
    "service_description": "I explain SAT math problems step-by-step and help improve accuracy.",
    "reason": "The request mentions SAT math word problems, which directly aligns with this service."
}
test_session = agent_3_polisher(test_session)

print("\nResult:")
print(test_session["final_response"])


Testing Agent 3: Polisher

🤖 AGENT 3: POLISHER is working...
   Reading matched service: SAT Math Tutor
   ✓ Final response created

Result:
Best Match: SAT Math Tutor
Description: I explain SAT math problems step-by-step and help improve accuracy.
Why: I saw you mentioned SAT math word problems, and that's exactly what this tutor specializes in! They can help you break down those problems and boost your confidence.


---

## 7. Pipeline Orchestration

### What is a Pipeline?

A **pipeline** is a sequence of operations where:
- Each operation receives input
- Each operation produces output
- The output of one operation becomes the input of the next

Think of it like an assembly line in a factory:
```
Raw materials → Station 1 → Station 2 → Station 3 → Finished product
```

Our pipeline:
```
User request → Interpreter → Matcher → Polisher → Final response
```

### Pipeline Pattern Benefits

1. **Sequential execution**: Steps happen in order (no parallelism needed here)
2. **Shared state**: The session dictionary flows through each step
3. **Composable**: Easy to add/remove/reorder agents
4. **Debuggable**: Can inspect session after any step

### The `run_pipeline()` Function

This is the orchestrator. It:
1. Creates a fresh session
2. Calls each agent in sequence
3. Passes the session from one to the next
4. Returns the final result

In [8]:
def run_pipeline(user_request: str, verbose: bool = True) -> str:
    """
    Orchestrates the multi-agent pipeline.
    
    This is the main entry point for the entire system. It:
    1. Creates a session with the user's request
    2. Runs each agent in sequence
    3. Returns the final polished response
    
    Args:
        user_request: The raw text from the user
        verbose: If True, print detailed progress information
        
    Returns:
        The final formatted response string
    """
    
    if verbose:
        print("\n" + "="*60)
        print("🚀 STARTING MULTI-AGENT PIPELINE")
        print("="*60)
        print(f"\nUser Request: \"{user_request}\"\n")
    
    # Step 1: Initialize session
    # The session is our "shared memory" that all agents read from and write to
    session = create_session(user_request)
    
    # Step 2: Run Agent 1 - Interpreter
    # Takes messy input, produces clean summary
    session = agent_1_interpreter(session)
    
    # Step 3: Run Agent 2 - Matcher
    # Takes clean summary, calls tool, produces matched service
    session = agent_2_matcher(session)
    
    # Step 4: Run Agent 3 - Polisher
    # Takes matched service, produces final user-friendly response
    session = agent_3_polisher(session)
    
    # Step 5: Extract and return the final result
    final_response = session["final_response"]
    
    if verbose:
        print("\n" + "="*60)
        print("✅ PIPELINE COMPLETE")
        print("="*60)
        # Optionally show the full session state for learning purposes
        # print_session_state(session, "Final Session State")
    
    return final_response


# Helper function to run and display results nicely
def match_service(user_request: str):
    """
    Convenience function that runs the pipeline and displays the result.
    """
    result = run_pipeline(user_request, verbose=True)
    print("\n" + "="*60)
    print("📋 FINAL RESPONSE")
    print("="*60)
    print("\n" + result + "\n")
    return result

---

## 8. Example Usage

Now let's test the complete system with various user requests!

**Note**: The examples below use mock responses. Once you configure your Gemini API key and uncomment the LLM calls in the agent functions, you'll see real AI-powered matching.

### Example 1: Python Debugging Request

In [9]:
match_service("my python code keeps crashing when i try to open a file help!!")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "my python code keeps crashing when i try to open a file help!!"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'my python code keeps crashing when i try to open a file help!!'
   ✓ Clarified to: 'My Python code crashes when attempting to open a file, and I need assistance.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'My Python code crashes when attempting to open a file, and I need assistance.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Python Debug Helper
   Reason: The user's problem is a crashing Python code, specifically related to file opening. The Python Debug Helper is designed to fix Python bugs, making it the most relevant service.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: Python Debug Helper
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Best Match: Python Debug Helper
Description: I help fix Python bugs quickly usi

"Best Match: Python Debug Helper\nDescription: I help fix Python bugs quickly using simple explanations.\nWhy: Your Python code is crashing when trying to open a file, which definitely sounds like a bug! The Python Debug Helper is here to help you squash that bug and get your code running smoothly. Let's figure out what's going wrong with that file opening!"

### Example 2: SAT Math Help

In [10]:
match_service("my kid needs help with sat math word problems")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "my kid needs help with sat math word problems"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'my kid needs help with sat math word problems'
   ✓ Clarified to: 'My child requires assistance with SAT math word problems.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'My child requires assistance with SAT math word problems.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: SAT Math Tutor
   Reason: The user specifically needs help with SAT math word problems, and the SAT Math Tutor service directly addresses this need by offering step-by-step explanations and aiming to improve accuracy in SAT math.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: SAT Math Tutor
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Okay, here's the response I've created:

Best Match: SAT Math Tutor
Description: I explain SAT math problems step-by-step and help impro

"Okay, here's the response I've created:\n\nBest Match: SAT Math Tutor\nDescription: I explain SAT math problems step-by-step and help improve accuracy.\nWhy: This looks like a great fit! You mentioned needing help with SAT math word problems, and this tutor focuses specifically on that by breaking down problems and helping you get those answers right!"

### Example 3: Tax Questions

In [11]:
match_service("i have questions about filing my taxes this year")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "i have questions about filing my taxes this year"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'i have questions about filing my taxes this year'
   ✓ Clarified to: 'Clarify questions about this year's tax filing process.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'Clarify questions about this year's tax filing process.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Tax Filing Advisor
   Reason: The user needs help with tax filing questions, and the Tax Filing Advisor service specifically addresses questions about tax forms and filing issues.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: Tax Filing Advisor
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Okay, here's the polished response:

Best Match: Tax Filing Advisor
Description: I answer questions about government tax forms and common filing issues.
Why: It sounds like you're 

"Okay, here's the polished response:\n\nBest Match: Tax Filing Advisor\nDescription: I answer questions about government tax forms and common filing issues.\nWhy: It sounds like you're looking for help with tax filing, and the Tax Filing Advisor is designed to answer questions about tax forms and any filing issues you might be running into! Let me know what's on your mind!"

### Example 4: Swimming Technique

In [13]:
match_service("I want to improve my freestyle stroke breathing")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "I want to improve my freestyle stroke breathing"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'I want to improve my freestyle stroke breathing'
   ✓ Clarified to: 'Clarify techniques for freestyle stroke breathing.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'Clarify techniques for freestyle stroke breathing.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Swimming Technique Review
   Reason: The user's request is about freestyle stroke breathing techniques, which directly aligns with the description of the Swimming Technique Review service.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: Swimming Technique Review
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Okay, here's a response tailored for the user:

Best Match: Swimming Technique Review
Description: I give fast feedback on stroke mechanics, breathing, and body position.
Why: 

"Okay, here's a response tailored for the user:\n\nBest Match: Swimming Technique Review\nDescription: I give fast feedback on stroke mechanics, breathing, and body position.\nWhy: This service sounds perfect for you! Since you're asking about freestyle stroke breathing techniques, I can give you some quick feedback on that, along with your overall stroke mechanics and body position. Let's get you swimming even better!"

### Example 5: Edge Case - Ambiguous Request

In [14]:
# This tests how the system handles requests that could match multiple services
match_service("I need help with numbers")


🚀 STARTING MULTI-AGENT PIPELINE

User Request: "I need help with numbers"


🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'I need help with numbers'
   ✓ Clarified to: 'The user requires assistance with numerical information.'

🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'The user requires assistance with numerical information.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: SAT Math Tutor
   Reason: The user needs assistance with numerical information, and the SAT Math Tutor focuses specifically on explaining and improving accuracy with math problems, making it the best match among the available services.

🤖 AGENT 3: POLISHER is working...
   Reading matched service: SAT Math Tutor
   ✓ Final response created

✅ PIPELINE COMPLETE

📋 FINAL RESPONSE

Okay, here's the polished response for the user:

Best Match: SAT Math Tutor
Description: I explain SAT math problems step-by-step and help improve accuracy.
Why: You mention

"Okay, here's the polished response for the user:\n\nBest Match: SAT Math Tutor\nDescription: I explain SAT math problems step-by-step and help improve accuracy.\nWhy: You mentioned needing help with numerical information, and the SAT Math Tutor is all about explaining math problems clearly and helping you get more accurate answers. Seems like a great fit!"

---

## 9. Debugging and Inspection

One of the benefits of the pipeline pattern is that you can inspect the session state at any point.

This is incredibly valuable for:
- **Learning**: See exactly what each agent produces
- **Debugging**: Find where things go wrong
- **Optimization**: Identify which agent needs improvement

In [15]:
def run_pipeline_with_inspection(user_request: str):
    """
    Run the pipeline but show session state after each agent.
    Useful for learning and debugging.
    """
    print("\n" + "="*60)
    print("🔍 RUNNING PIPELINE WITH INSPECTION")
    print("="*60)
    
    session = create_session(user_request)
    print_session_state(session, "Initial State")
    
    session = agent_1_interpreter(session)
    print_session_state(session, "After Agent 1 (Interpreter)")
    
    session = agent_2_matcher(session)
    print_session_state(session, "After Agent 2 (Matcher)")
    
    session = agent_3_polisher(session)
    print_session_state(session, "After Agent 3 (Polisher)")
    
    return session["final_response"]

# Example: Inspect a request step-by-step
run_pipeline_with_inspection("python loop broken help")


🔍 RUNNING PIPELINE WITH INSPECTION

  Initial State

user_request:
  python loop broken help

clarified_request:
  None

matched_service:
  None

final_response:
  None



🤖 AGENT 1: INTERPRETER is working...
   Reading user request: 'python loop broken help'
   ✓ Clarified to: 'The user needs help debugging a broken loop in their Python code.'

  After Agent 1 (Interpreter)

user_request:
  python loop broken help

clarified_request:
  The user needs help debugging a broken loop in their Python code.

matched_service:
  None

final_response:
  None



🤖 AGENT 2: MATCHER is working...
   Reading clarified request: 'The user needs help debugging a broken loop in their Python code.'
   Calling get_services() tool...
   ✓ Retrieved 4 services
   ✓ Matched to: Python Debug Helper
   Reason: The user specifically needs help debugging Python code, and the Python Debug Helper is designed for that purpose.

  After Agent 2 (Matcher)

user_request:
  python loop broken help

clarified_request:

"Okay, here's the response:\n\nBest Match: Python Debug Helper\nDescription: I help fix Python bugs quickly using simple explanations.\nWhy: You're looking for help debugging your Python code, and that's exactly what I'm designed to do! I'll help you squash those bugs!"

---

## 10. Key Takeaways and Learning Summary

Congratulations! You've built a complete multi-agent system. Let's review what you learned:

### 1. Multi-Agent Systems

**What**: Multiple AI agents working together, each with a specific role

**Why**: 
- Separation of concerns (each agent has one job)
- Easier to debug and improve
- More modular and maintainable

**How**: 
- Define clear roles for each agent
- Use prompts to give each agent its "personality"
- Chain agents in a sequence (pipeline pattern)

### 2. Custom Tools

**What**: Functions that agents can call to access data or perform actions

**Why**: 
- LLMs can't access external data without tools
- Tools provide reliable, structured information
- Tools extend what agents can do beyond text generation

**How**: 
- Create simple functions that return data
- Call tools in your agent code
- Include tool output in prompts for LLM reasoning

### 3. Session State Management

**What**: A shared memory space where agents store and retrieve information

**Why**: 
- Agents need to communicate
- Each agent builds on previous work
- Enables sequential processing

**How**: 
- Use a dictionary to store state
- Each agent reads from and writes to the session
- Pass the session through the pipeline

### 4. Pipeline Pattern

**What**: A sequence of operations where output flows from one to the next

**Why**: 
- Simple and predictable execution
- Easy to reason about
- Easy to extend or modify

**How**: 
- Create functions for each stage
- Each function takes state, modifies it, returns it
- Orchestrate with a main function

---

## Next Steps

To deepen your learning, try these exercises:

1. **Add a 4th agent**: Create a "Validator" that checks if the match quality is high enough
   
2. **Add more services**: Expand the service list to 10+ items and test edge cases
   
3. **Add a second tool**: Create `get_user_history()` that returns past requests, and use it in Agent 2
   
4. **Improve prompts**: Experiment with different prompt styles to improve matching accuracy
   
5. **Add error handling**: What happens if the LLM returns invalid JSON? Add try/catch blocks
   
6. **Parallel agents**: Research how to run multiple agents in parallel instead of sequentially
   
7. **Add confidence scores**: Have Agent 2 return a confidence score (0-100) with each match

---

## Resources for Further Learning

- **LangChain Documentation**: More advanced agent frameworks
- **AutoGen**: Microsoft's multi-agent conversation framework  
- **CrewAI**: Role-based agent collaboration
- **OpenAI Function Calling**: How to let LLMs decide which tools to use

---

**Remember**: The best way to learn is by building. Take this notebook and modify it. Break it. Fix it. Add features. That's how you truly understand agentic systems!

Happy coding! 🚀